# **Chapter 26: Desktop & Web Support**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Configure Flutter applications for Windows, macOS, and Linux desktop platforms
- Implement responsive layouts that adapt to desktop screen sizes and input methods
- Handle keyboard shortcuts, focus management, and mouse interactions specific to desktop
- Deploy Flutter web applications with proper PWA (Progressive Web App) configuration
- Handle CORS (Cross-Origin Resource Sharing) issues in web deployments
- Choose between CanvasKit and HTML renderers based on application requirements
- Optimize performance for web and desktop targets

---

## **Prerequisites**

- Completed Chapter 9: Layout and Composition (understanding of responsive design)
- Completed Chapter 10: Input, Forms, and Validation (FocusNode, TextField)
- Completed Chapter 23: Platform Integration (platform channel concepts)
- Basic understanding of web technologies (HTML, CSS, JavaScript)
- Familiarity with browser developer tools and debugging
- For desktop: Access to Windows, macOS, or Linux development environment

---

## **26.1 Desktop Platform Setup**

Flutter desktop support allows building native applications for Windows, macOS, and Linux from a single codebase. Unlike mobile, desktop apps run in resizable windows with different input paradigms.

### **Platform Configuration**

```dart
// lib/utils/platform_utils.dart
// Utilities for detecting and handling desktop platforms

import 'dart:io';
import 'package:flutter/foundation.dart';
import 'package:flutter/material.dart';
import 'package:window_manager/window_manager.dart';  // package:window_manager

/// Desktop-specific utilities and window management
class DesktopUtils {
  /// Check if running on any desktop platform
  static bool get isDesktop {
    return Platform.isWindows || Platform.isMacOS || Platform.isLinux;
  }
  
  /// Check if running on web
  static bool get isWeb => kIsWeb;
  
  /// Check if running on mobile
  static bool get isMobile => !isDesktop && !isWeb;
  
  /// Initialize desktop window settings
  /// Call this in main() before runApp() for desktop platforms
  static Future<void> initializeWindow() async {
    if (!isDesktop) return;
    
    // Must initialize window manager
    await windowManager.ensureInitialized();
    
    WindowOptions windowOptions = const WindowOptions(
      size: Size(1200, 800),        // Default window size
      minimumSize: Size(800, 600),  // Prevent resizing too small
      center: true,                 // Center on screen
      backgroundColor: Colors.transparent,
      skipTaskbar: false,
      titleBarStyle: TitleBarStyle.hidden,  // Custom title bar (optional)
    );
    
    await windowManager.waitUntilReadyToShow(windowOptions, () async {
      await windowManager.show();
      await windowManager.focus();
    });
  }
  
  /// Set window title dynamically
  static Future<void> setWindowTitle(String title) async {
    if (isDesktop) {
      await windowManager.setTitle(title);
    }
  }
  
  /// Enter fullscreen mode
  static Future<void> setFullscreen(bool fullscreen) async {
    if (isDesktop) {
      if (fullscreen) {
        await windowManager.setFullScreen();
      } else {
        await windowManager.setFullScreen(false);
      }
    }
  }
  
  /// Minimize window to taskbar/dock
  static Future<void> minimize() async {
    if (isDesktop) {
      await windowManager.minimize();
    }
  }
  
  /// Check if window is focused
  static Future<bool> isFocused() async {
    if (!isDesktop) return true;
    return await windowManager.isFocused();
  }
}

/// Widget that listens to window focus changes
/// Useful for pausing/resuming animations or sync
class WindowFocusListener extends StatefulWidget {
  final Widget child;
  final VoidCallback? onFocus;
  final VoidCallback? onBlur;
  
  const WindowFocusListener({
    Key? key,
    required this.child,
    this.onFocus,
    this.onBlur,
  }) : super(key: key);
  
  @override
  _WindowFocusListenerState createState() => _WindowFocusListenerState();
}

class _WindowFocusListenerState extends State<WindowFocusListener> with WindowListener {
  bool _isFocused = true;
  
  @override
  void initState() {
    super.initState();
    windowManager.addListener(this);
  }
  
  @override
  void dispose() {
    windowManager.removeListener(this);
    super.dispose();
  }
  
  @override
  void onWindowFocus() {
    setState(() => _isFocused = true);
    widget.onFocus?.call();
  }
  
  @override
  void onWindowBlur() {
    setState(() => _isFocused = false);
    widget.onBlur?.call();
  }
  
  @override
  Widget build(BuildContext context) {
    return widget.child;
  }
}
```

**Explanation:**

- **window_manager**: A community package that provides control over desktop window properties (size, position, fullscreen, minimization). The official Flutter desktop API is limited, so this package fills critical gaps.
- **TitleBarStyle.hidden**: Allows creating custom title bars (like Spotify or VS Code) instead of using the native OS chrome. You must then implement your own drag-to-move and window controls.
- **WindowListener**: Mixin that provides callbacks for window events (focus, blur, resize, close). Critical for desktop apps to pause expensive operations (animations, video playback) when window loses focus.
- **Platform Detection**: `Platform.isWindows` throws on web, so always check `kIsWeb` first or use conditional imports. The helper methods provide safe shortcuts.

### **Custom Title Bar (Frameless Window)**

```dart
// lib/widgets/desktop_title_bar.dart
// Custom title bar for frameless desktop windows

import 'package:flutter/material.dart';
import 'package:window_manager/window_manager.dart';

class DesktopTitleBar extends StatelessWidget {
  final Widget? title;
  final List<Widget>? actions;
  final Color? backgroundColor;
  
  const DesktopTitleBar({
    Key? key,
    this.title,
    this.actions,
    this.backgroundColor,
  }) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    // Only show on desktop
    if (!DesktopUtils.isDesktop) return SizedBox.shrink();
    
    return Container(
      height: 40,
      color: backgroundColor ?? Theme.of(context).appBarTheme.backgroundColor,
      child: Row(
        children: [
          // Draggable area (moves window when dragged)
          Expanded(
            child: GestureDetector(
              behavior: HitTestBehavior.translucent,
              onPanStart: (_) => windowManager.startDragging(),
              onDoubleTap: () async {
                if (await windowManager.isMaximized()) {
                  windowManager.unmaximize();
                } else {
                  windowManager.maximize();
                }
              },
              child: Container(
                alignment: Alignment.centerLeft,
                padding: EdgeInsets.symmetric(horizontal: 16),
                child: DefaultTextStyle(
                  style: TextStyle(
                    fontSize: 14,
                    fontWeight: FontWeight.w600,
                    color: Theme.of(context).appBarTheme.foregroundColor,
                  ),
                  child: title ?? Text('My Desktop App'),
                ),
              ),
            ),
          ),
          
          // Custom actions
          if (actions != null) ...actions!,
          
          // Window controls
          _WindowControlButton(
            icon: Icons.remove,
            onPressed: () => windowManager.minimize(),
            tooltip: 'Minimize',
          ),
          _WindowControlButton(
            icon: Icons.crop_square,
            onPressed: () async {
              if (await windowManager.isMaximized()) {
                windowManager.unmaximize();
              } else {
                windowManager.maximize();
              }
            },
            tooltip: 'Maximize',
          ),
          _WindowControlButton(
            icon: Icons.close,
            onPressed: () => windowManager.close(),
            tooltip: 'Close',
            hoverColor: Colors.red,
          ),
        ],
      ),
    );
  }
}

class _WindowControlButton extends StatefulWidget {
  final IconData icon;
  final VoidCallback onPressed;
  final String tooltip;
  final Color? hoverColor;
  
  const _WindowControlButton({
    required this.icon,
    required this.onPressed,
    required this.tooltip,
    this.hoverColor,
  });
  
  @override
  __WindowControlButtonState createState() => __WindowControlButtonState();
}

class __WindowControlButtonState extends State<_WindowControlButton> {
  bool _isHovering = false;
  
  @override
  Widget build(BuildContext context) {
    return MouseRegion(
      onEnter: (_) => setState(() => _isHovering = true),
      onExit: (_) => setState(() => _isHovering = false),
      child: GestureDetector(
        onTap: widget.onPressed,
        child: Container(
          width: 46,
          height: 40,
          color: _isHovering ? (widget.hoverColor ?? Colors.white.withOpacity(0.1)) : Colors.transparent,
          child: Icon(
            widget.icon,
            size: 20,
            color: Theme.of(context).appBarTheme.foregroundColor,
          ),
        ),
      ),
    );
  }
}
```

**Explanation:**

- **GestureDetector with startDragging()**: Makes the title bar draggable. Without this, users cannot move frameless windows. `onDoubleTap` toggles maximize/restore.
- **MouseRegion**: Essential for desktop hover states. Unlike mobile, desktop users expect visual feedback when hovering over interactive elements (buttons, menus).
- **Maximize Detection**: Window state is asynchronous (`isMaximized()` returns Future). Use `WindowListener` to track state changes in real-time if you need to update the icon (show restore icon when maximized).

---

## **26.2 Responsive Desktop Layouts**

Desktop apps require layouts that adapt to wide screens, support resizable windows, and handle different input methods (mouse, trackpad, keyboard).

### **Adaptive Layout Architecture**

```dart
// lib/layout/adaptive_scaffold.dart
// Responsive scaffold that adapts to desktop, tablet, and mobile

import 'package:flutter/material.dart';

/// Breakpoints for responsive design
class Breakpoints {
  static const double mobile = 600;
  static const double tablet = 1024;
  static const double desktop = 1440;
  
  static bool isMobile(BuildContext context) => 
      MediaQuery.of(context).size.width < mobile;
  static bool isTablet(BuildContext context) => 
      MediaQuery.of(context).size.width >= mobile && 
      MediaQuery.of(context).size.width < tablet;
  static bool isDesktop(BuildContext context) => 
      MediaQuery.of(context).size.width >= tablet;
}

/// Adaptive scaffold with navigation that changes based on screen size
class AdaptiveScaffold extends StatefulWidget {
  final Widget title;
  final List<NavigationDestination> destinations;
  final Widget body;
  final int selectedIndex;
  final Function(int) onDestinationSelected;
  final List<Widget>? actions;
  
  const AdaptiveScaffold({
    Key? key,
    required this.title,
    required this.destinations,
    required this.body,
    required this.selectedIndex,
    required this.onDestinationSelected,
    this.actions,
  }) : super(key: key);
  
  @override
  _AdaptiveScaffoldState createState() => _AdaptiveScaffoldState();
}

class _AdaptiveScaffoldState extends State<AdaptiveScaffold> {
  @override
  Widget build(BuildContext context) {
    // Desktop: Side navigation rail (permanent)
    if (Breakpoints.isDesktop(context)) {
      return Scaffold(
        body: Row(
          children: [
            NavigationRail(
              extended: MediaQuery.of(context).size.width > Breakpoints.desktop,
              minExtendedWidth: 200,
              selectedIndex: widget.selectedIndex,
              onDestinationSelected: widget.onDestinationSelected,
              leading: Padding(
                padding: EdgeInsets.all(16),
                child: widget.title,
              ),
              trailing: widget.actions != null 
                  ? Column(
                      mainAxisAlignment: MainAxisAlignment.end,
                      children: widget.actions!,
                    )
                  : null,
              destinations: widget.destinations.map((d) => 
                NavigationRailDestination(
                  icon: d.icon,
                  selectedIcon: d.selectedIcon,
                  label: Text(d.label),
                )
              ).toList(),
            ),
            VerticalDivider(thickness: 1, width: 1),
            Expanded(child: widget.body),
          ],
        ),
      );
    }
    
    // Tablet: Modal side drawer or rail
    if (Breakpoints.isTablet(context)) {
      return Scaffold(
        appBar: AppBar(
          title: widget.title,
          actions: widget.actions,
        ),
        body: Row(
          children: [
            NavigationRail(
              selectedIndex: widget.selectedIndex,
              onDestinationSelected: widget.onDestinationSelected,
              destinations: widget.destinations.map((d) => 
                NavigationRailDestination(
                  icon: d.icon,
                  selectedIcon: d.selectedIcon,
                  label: Text(d.label),
                )
              ).toList(),
            ),
            VerticalDivider(thickness: 1, width: 1),
            Expanded(child: widget.body),
          ],
        ),
      );
    }
    
    // Mobile: Bottom navigation bar
    return Scaffold(
      appBar: AppBar(
        title: widget.title,
        actions: widget.actions,
      ),
      body: widget.body,
      bottomNavigationBar: NavigationBar(
        selectedIndex: widget.selectedIndex,
        onDestinationSelected: widget.onDestinationSelected,
        destinations: widget.destinations,
      ),
    );
  }
}

/// Data class for navigation destinations
class NavigationDestination {
  final Widget icon;
  final Widget? selectedIcon;
  final String label;
  
  NavigationDestination({
    required this.icon,
    this.selectedIcon,
    required this.label,
  });
}

/// Master-detail layout for desktop (e.g., email client)
class MasterDetailLayout extends StatelessWidget {
  final Widget master;      // List/sidebar
  final Widget? detail;     // Detail view (null = empty state)
  final double masterWidth;
  final double breakpoint;
  
  const MasterDetailLayout({
    Key? key,
    required this.master,
    this.detail,
    this.masterWidth = 360,
    this.breakpoint = 700,
  }) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    final width = MediaQuery.of(context).size.width;
    
    if (width < breakpoint) {
      // Mobile: Show master or detail, not both
      return Navigator(
        onGenerateRoute: (settings) {
          return MaterialPageRoute(
            builder: (context) => master,
          );
        },
      );
    }
    
    // Desktop/Tablet: Side-by-side
    return Row(
      children: [
        Container(
          width: masterWidth,
          decoration: BoxDecoration(
            border: Border(
              right: BorderSide(color: Theme.of(context).dividerColor),
            ),
          ),
          child: master,
        ),
        Expanded(
          child: detail ?? Center(
            child: Text(
              'Select an item to view details',
              style: Theme.of(context).textTheme.headlineSmall,
            ),
          ),
        ),
      ],
    );
  }
}
```

**Explanation:**

- **NavigationRail**: The desktop standard for side navigation. Use `extended: true` when width allows to show labels next to icons. `minExtendedWidth` prevents rail from becoming too wide.
- **Breakpoints**: Material Design 3 recommends 600dp for mobile/tablet split and 1024dp for tablet/desktop. Adjust based on your content (data tables need more width than lists).
- **Master-Detail**: Common pattern in desktop apps (mail, file explorers). On mobile, use navigation; on desktop, show both panes. Provide empty state when no item selected.
- **VerticalDivider**: Creates visual separation between navigation and content. Use `width: 1` to ensure it doesn't take extra space beyond the 1px line.

---

## **26.3 Keyboard and Mouse Interactions**

Desktop users expect rich keyboard shortcuts, focus navigation, and hover interactions.

### **Keyboard Shortcuts and Focus**

```dart
// lib/shortcuts/app_shortcuts.dart
// Global keyboard shortcuts for desktop apps

import 'package:flutter/material.dart';
import 'package:flutter/services.dart';

/// Intent classes for actions
class NewDocumentIntent extends Intent {}
class SaveIntent extends Intent {}
class CloseWindowIntent extends Intent {}
class SearchIntent extends Intent {}

/// Shortcuts configuration
class AppShortcuts extends StatelessWidget {
  final Widget child;
  
  const AppShortcuts({Key? key, required this.child}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    // Only enable shortcuts on desktop/web
    if (DesktopUtils.isMobile) return child;
    
    return Shortcuts(
      shortcuts: {
        // Ctrl/Cmd + N
        SingleActivator(LogicalKeyboardKey.keyN, meta: true): NewDocumentIntent(),
        SingleActivator(LogicalKeyboardKey.keyN, control: true): NewDocumentIntent(),
        
        // Ctrl/Cmd + S
        SingleActivator(LogicalKeyboardKey.keyS, meta: true): SaveIntent(),
        SingleActivator(LogicalKeyboardKey.keyS, control: true): SaveIntent(),
        
        // Ctrl/Cmd + W (close window)
        SingleActivator(LogicalKeyboardKey.keyW, meta: true): CloseWindowIntent(),
        SingleActivator(LogicalKeyboardKey.keyW, control: true): CloseWindowIntent(),
        
        // Ctrl/Cmd + F (search)
        SingleActivator(LogicalKeyboardKey.keyF, meta: true): SearchIntent(),
        SingleActivator(LogicalKeyboardKey.keyF, control: true): SearchIntent(),
        
        // Escape
        SingleActivator(LogicalKeyboardKey.escape): CloseWindowIntent(),
      },
      child: Actions(
        actions: {
          NewDocumentIntent: CallbackAction<NewDocumentIntent>(
            onInvoke: (intent) {
              print('New document');
              return null;
            },
          ),
          SaveIntent: CallbackAction<SaveIntent>(
            onInvoke: (intent) {
              print('Save document');
              return null;
            },
          ),
          CloseWindowIntent: CallbackAction<CloseWindowIntent>(
            onInvoke: (intent) {
              if (DesktopUtils.isDesktop) {
                windowManager.close();
              }
              return null;
            },
          ),
          SearchIntent: CallbackAction<SearchIntent>(
            onInvoke: (intent) {
              // Focus search field
              return null;
            },
          ),
        },
        child: child,
      ),
    );
  }
}

/// Focus traversal configuration for desktop
/// Improves Tab navigation order
class DesktopFocusTraversal extends StatelessWidget {
  final Widget child;
  
  const DesktopFocusTraversal({Key? key, required this.child}) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    return FocusTraversalGroup(
      policy: WidgetOrderTraversalPolicy(),
      child: child,
    );
  }
}

/// Tooltip wrapper for desktop (shows on hover)
/// On mobile, tooltips show on long-press
class DesktopTooltip extends StatelessWidget {
  final String message;
  final Widget child;
  final Widget? richMessage;
  
  const DesktopTooltip({
    Key? key,
    required this.message,
    required this.child,
    this.richMessage,
  }) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    // Disable tooltips on mobile to avoid clutter
    if (DesktopUtils.isMobile) return child;
    
    return Tooltip(
      message: message,
      richMessage: richMessage,
      waitDuration: Duration(milliseconds: 500),  // Faster than default
      showDuration: Duration(seconds: 5),
      child: child,
    );
  }
}

/// Context menu for right-click (desktop) / long-press (mobile)
class AdaptiveContextMenu extends StatelessWidget {
  final Widget child;
  final List<PopupMenuEntry> items;
  
  const AdaptiveContextMenu({
    Key? key,
    required this.child,
    required this.items,
  }) : super(key: key);
  
  @override
  Widget build(BuildContext context) {
    if (DesktopUtils.isDesktop) {
      // Desktop: Right-click context menu
      return GestureDetector(
        onSecondaryTapUp: (details) {
          _showMenu(context, details.globalPosition);
        },
        child: child,
      );
    }
    
    // Mobile: Long press
    return GestureDetector(
      onLongPress: () {
        _showMenu(context, null);
      },
      child: child,
    );
  }
  
  void _showMenu(BuildContext context, Offset? position) {
    showMenu(
      context: context,
      position: position != null 
          ? RelativeRect.fromLTRB(
              position.dx, 
              position.dy, 
              position.dx + 1, 
              position.dy + 1,
            )
          : null,
      items: items,
    );
  }
}
```

**Explanation:**

- **SingleActivator**: Defines a keyboard shortcut. `meta` is Command on macOS, Windows key on Windows. `control` is Ctrl. You must define both to support all platforms, or use `meta: !Platform.isWindows` logic.
- **Shortcuts Widget**: Maps key combinations to Intents. Must be above `MaterialApp` or at specific scopes (like a text editor) to work.
- **Actions Widget**: Maps Intents to handlers. The `CallbackAction` executes the callback when the intent is invoked.
- **onSecondaryTapUp**: Right-click on desktop (mouse secondary button). `details.globalPosition` gives coordinates for positioning the context menu exactly where clicked.
- **FocusTraversalGroup**: Controls Tab order. By default, Flutter traverses widgets in widget tree order, which may not match visual order. Explicit traversal policies improve keyboard navigation accessibility.

---

## **26.4 Web Deployment and Configuration**

Flutter web requires specific configuration for hosting, CORS, PWA support, and renderer selection.

### **Web Configuration and CORS**

```dart
// lib/services/web_api_service.dart
// Handling CORS and web-specific API calls

import 'package:flutter/foundation.dart';
import 'dart:html' as html;  // Only available on web
import 'dart:js' as js;      // JavaScript interop

class WebApiService {
  /// Make CORS-enabled request from web
  /// Standard http package may fail due to CORS policy
  static Future<String> fetchWithCORS(String url) async {
    if (!kIsWeb) {
      throw UnsupportedError('Only for web platform');
    }
    
    // Use browser's fetch API which handles CORS properly
    final response = await html.HttpRequest.request(
      url,
      method: 'GET',
      requestHeaders: {
        'Content-Type': 'application/json',
        // CORS headers must be allowed by server
      },
      withCredentials: false,  // true if sending cookies/auth
    );
    
    return response.responseText ?? '';
  }
  
  /// Download file on web (triggers browser download)
  static void downloadFile(String url, String filename) {
    if (!kIsWeb) return;
    
    final anchor = html.AnchorElement(href: url)
      ..setAttribute('download', filename)
      ..style.display = 'none';
    
    html.document.body?.children.add(anchor);
    anchor.click();
    anchor.remove();
  }
  
  /// Open new tab
  static void openInNewTab(String url) {
    if (!kIsWeb) return;
    html.window.open(url, '_blank');
  }
  
  /// Get current URL parameters
  static Map<String, String> getUrlParameters() {
    if (!kIsWeb) return {};
    
    final uri = Uri.parse(html.window.location.href);
    return uri.queryParameters;
  }
  
  /// Update browser URL without reloading (SPA navigation)
  static void updateUrl(String path, {Map<String, String>? queryParams}) {
    if (!kIsWeb) return;
    
    final uri = Uri(
      path: path,
      queryParameters: queryParams,
    );
    
    html.window.history.pushState(null, '', uri.toString());
  }
  
  /// Listen to browser back/forward buttons
  static void listenToPopState(Function(String) onPathChanged) {
    if (!kIsWeb) return;
    
    html.window.onPopState.listen((event) {
      onPathChanged(html.window.location.pathname ?? '/');
    });
  }
  
  /// Check if running in PWA mode (installed)
  static bool isRunningAsPWA() {
    if (!kIsWeb) return false;
    // Check display-mode: standalone via CSS media query
    return html.window.matchMedia('(display-mode: standalone)').matches ||
           js.context.hasProperty('webkit') && 
           js.context['webkit'].hasProperty('standalone');
  }
  
  /// Store data in localStorage (persistent)
  static void setLocalStorage(String key, String value) {
    if (!kIsWeb) return;
    html.window.localStorage[key] = value;
  }
  
  static String? getLocalStorage(String key) {
    if (!kIsWeb) return null;
    return html.window.localStorage[key];
  }
  
  /// Store data in sessionStorage (tab-specific)
  static void setSessionStorage(String key, String value) {
    if (!kIsWeb) return;
    html.window.sessionStorage[key] = value;
  }
}
```

**Explanation:**

- **dart:html**: Only imported when compiling to web. Use conditional imports (`import 'dart:html' if (dart.library.io) '...'`) for cross-platform files, or separate web-specific implementations.
- **CORS**: Browsers block cross-origin requests unless the server sends proper `Access-Control-Allow-Origin` headers. `html.HttpRequest` uses the browser's native fetch which handles preflight OPTIONS requests automatically.
- **localStorage/sessionStorage**: Web storage APIs. localStorage persists until explicitly cleared; sessionStorage clears when tab closes. 5-10MB limit typically.
- **PWA Detection**: Checks if app is running in standalone mode (installed from browser). Useful to show "Install app" prompts only when not installed.
- **History API**: `pushState` updates the URL without page reload, enabling Single Page Application (SPA) behavior with proper deep linking.

### **index.html Configuration**

```html
<!-- web/index.html -->

<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta content="IE=Edge" http-equiv="X-UA-Compatible">
  
  <!-- Viewport for responsive design -->
  <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no">
  
  <!-- PWA Manifest -->
  <link rel="manifest" href="manifest.json">
  
  <!-- Theme Color for browser chrome -->
  <meta name="theme-color" content="#1976d2">
  
  <!-- iOS Meta Tags -->
  <meta name="apple-mobile-web-app-capable" content="yes">
  <meta name="apple-mobile-web-app-status-bar-style" content="black-translucent">
  <meta name="apple-mobile-web-app-title" content="My Flutter App">
  <link rel="apple-touch-icon" href="icons/Icon-192.png">
  
  <!-- Favicon -->
  <link rel="icon" type="image/png" href="favicon.png"/>
  
  <title>My Flutter App</title>
  
  <!-- Loading indicator styles -->
  <style>
    body {
      margin: 0;
      background-color: #1976d2;
    }
    .loading {
      position: absolute;
      top: 50%;
      left: 50%;
      transform: translate(-50%, -50%);
      color: white;
      font-family: sans-serif;
    }
  </style>
  
  <script>
    // The value below is injected by flutter build, do not touch.
    const serviceWorkerVersion = null;
  </script>
  
  <!-- This script adds the flutter initialization JS code -->
  <script src="flutter.js" defer></script>
</head>
<body>
  <!-- Loading indicator shown before Flutter loads -->
  <div id="loading" class="loading">
    <h2>Loading...</h2>
  </div>
  
  <script>
    window.addEventListener('load', function(ev) {
      // Download main.dart.js
      _flutter.loader.loadEntrypoint({
        serviceWorker: {
          serviceWorkerVersion: serviceWorkerVersion,
        },
        onEntrypointLoaded: function(engineInitializer) {
          // Initialize engine
          engineInitializer.initializeEngine().then(function(appRunner) {
            // Remove loading indicator
            document.getElementById('loading').remove();
            // Run app
            appRunner.runApp();
          });
        }
      });
    });
  </script>
</body>
</html>
```

**Explanation:**

- **Viewport**: `user-scalable=no` prevents pinch-to-zoom on mobile web, behaving more like a native app. Remove if text scaling is important for accessibility.
- **Manifest**: JSON file defining app name, icons, display mode (standalone/fullscreen), and theme colors. Required for PWA install prompt.
- **Service Worker**: Caches assets for offline functionality. `flutter.js` handles service worker registration automatically with the provided `serviceWorkerVersion`.
- **Theme Color**: Colors the browser address bar on mobile Chrome/Safari to match your app theme.

### **PWA Manifest**

```json
{
  "name": "My Flutter Desktop App",
  "short_name": "FlutterApp",
  "start_url": ".",
  "display": "standalone",
  "background_color": "#1976d2",
  "theme_color": "#1976d2",
  "description": "A desktop-class web application built with Flutter",
  "orientation": "any",
  "prefer_related_applications": false,
  "icons": [
    {
      "src": "icons/Icon-192.png",
      "sizes": "192x192",
      "type": "image/png"
    },
    {
      "src": "icons/Icon-512.png",
      "sizes": "512x512",
      "type": "image/png",
      "purpose": "any maskable"
    }
  ]
}
```

---

## **26.5 Renderer Selection (CanvasKit vs HTML)**

Flutter web offers two rendering backends with different trade-offs.

### **Renderer Configuration**

```dart
// lib/main.dart
// Runtime renderer detection and adaptation

import 'package:flutter/foundation.dart';
import 'package:flutter/material.dart';

void main() {
  // Auto-select renderer based on device type
  // CanvasKit: Better for desktop, complex UI, WebGL
  // HTML: Better for mobile web, text-heavy, smaller download
  
  if (kIsWeb) {
    // Detected at runtime via JavaScript
    final isMobileBrowser = _isMobileBrowser();
    
    // Force specific renderer via build command:
    // flutter run -d chrome --web-renderer html
    // flutter build web --web-renderer canvaskit
    
    print('Running on mobile browser: $isMobileBrowser');
  }
  
  runApp(MyApp());
}

bool _isMobileBrowser() {
  // This would need js interop in real implementation
  // Simplified example
  return false;
}

/// Widget that adapts based on renderer capabilities
class RendererAdaptiveWidget extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    // Check if CanvasKit (full Flutter engine)
    final isCanvasKit = _isCanvasKitRenderer();
    
    if (isCanvasKit) {
      // Use ShaderMask, ColorFilter, complex animations
      return CanvasKitOptimizedWidget();
    } else {
      // Use simpler visuals for HTML renderer
      // Avoids ShaderMask, some blend modes, complex paths
      return HtmlOptimizedWidget();
    }
  }
  
  bool _isCanvasKitRenderer() {
    if (!kIsWeb) return true;  // Mobile/desktop always "full" renderer
    // Check via js interop: window.flutterCanvasKit != null
    return true;  // Placeholder
  }
}

/// Performance considerations for web
class WebPerformance {
  /// Defer heavy initialization on web
  static void deferredInitialization(VoidCallback init) {
    if (kIsWeb) {
      // Use SchedulerBinding to run after first frame
      WidgetsBinding.instance.addPostFrameCallback((_) {
        init();
      });
    } else {
      init();
    }
  }
  
  /// Use Image.network with caching for web
  static Widget optimizedImage(String url) {
    if (kIsWeb) {
      // HTML renderer handles img tags better
      // CanvasKit prefers bundled assets or CORS-enabled images
      return Image.network(
        url,
        headers: {'Access-Control-Allow-Origin': '*'},
        errorBuilder: (context, error, stack) => Icon(Icons.error),
      );
    }
    return Image.network(url);
  }
}
```

**Explanation:**

- **CanvasKit**: Compiles Skia (Flutter's graphics engine) to WebAssembly. Provides pixel-perfect consistency with mobile/desktop Flutter. Supports all Flutter features (shaders, complex paths, text layout). Downside: Larger initial download (~2MB WASM) and slower startup.
- **HTML**: Uses standard HTML/CSS/DOM to render. Smaller download, faster startup, better SEO (text is selectable/accessible). Downside: Missing some visual effects (ShaderMask, some blend modes), inconsistent text rendering across browsers.
- **Auto Selection**: Flutter defaults to HTML on mobile browsers, CanvasKit on desktop browsers. You can override with `--web-renderer` flag.
- **CORS Images**: CanvasKit requires images to have CORS headers or be served from same origin. HTML renderer is more lenient with cross-origin images.

---

## **Chapter Summary**

In this chapter, we covered Flutter's desktop and web platform support:

### **Key Takeaways:**

1. **Desktop Windows**: Use `window_manager` for window control (size, minimize, maximize, custom title bars). Implement `WindowListener` to handle focus changes and window events. Use frameless windows with custom title bars for branded experiences.

2. **Responsive Layouts**: Use `NavigationRail` for desktop side navigation, `BottomNavigationBar` for mobile. Implement master-detail layouts that show both panes on desktop, navigate on mobile. Use breakpoints at 600dp and 1024dp.

3. **Input Handling**: Desktop requires hover states (`MouseRegion`), right-click context menus (`onSecondaryTapUp`), and keyboard shortcuts (`Shortcuts` + `Actions`). Implement focus traversal for keyboard navigation accessibility.

4. **Web CORS**: Use `dart:html` for web-specific APIs like `HttpRequest` to handle CORS properly. Use `localStorage`/`sessionStorage` for persistence. Implement History API for SPA navigation without page reloads.

5. **PWA Configuration**: Configure `manifest.json` for installability. Provide 192px and 512px icons. Set `display: standalone` for native-like experience. Service workers cache assets for offline functionality.

6. **Renderer Choice**: CanvasKit provides full Flutter fidelity but large download size (~2MB). HTML renderer is lighter and faster but lacks some visual effects. Use CanvasKit for desktop web apps, HTML for content-heavy mobile web.

### **Performance Considerations:**

- **Desktop**: Use `DeferredLoading` for heavy assets. Implement window focus detection to pause animations when app is not visible.
- **Web**: Compress images aggressively. Use `Image.network` with proper caching headers. Defer non-critical initialization using `addPostFrameCallback`.
- **Bundle Size**: Tree-shake unused code. Use `--split-debug-info` and `--obfuscate` for production web builds to reduce JavaScript size.

### **Next Steps:**

The next chapter (Chapter 27) will cover **Testing & Quality Assurance**, including:
- Unit testing with mockito/mocktail
- Widget testing with finders and matchers
- Integration testing across platforms
- Golden tests for visual regression
- Performance testing and profiling
- CI/CD integration for automated testing

---

**End of Chapter 26**